In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import openpyxl
import datetime
import re
import string
from bs4 import BeautifulSoup
import requests


In [373]:
URL ='/Users/veesheenyuen/Desktop/DataScience/SAA/Tilastopaja/Tilastopaja - 17th Asian Games - Incheon.html'  # open a locally saved copy of html
HTMLFileToBeOpened = open(URL, "r") 
contents = HTMLFileToBeOpened.read() 
soup = BeautifulSoup(contents, 'lxml') 


In [374]:
print(soup.body.prettify()) 


<body onload="document.getElementById('home').className='active'; leader(); SetLoginFocus();">
 <script>
  function load(){
	document.getElementById("siteLoader").style.display="block";
}
 </script>
 <div class="wrapper">
  <div class="top">
   <div class="logo">
    <h1>
     <a ref="https://www.tilastopaja.info">
      TILASTOPAJA
     </a>
    </h1>
   </div>
   <div class="container">
    <ul id="nav">
     <li class="haku">
      <div>
       SEARCH FOR AN ATHLETE
       <form action="https://www.tilastopaja.info/db/ats.php" method="GET">
        <input class="text" name="Name" size="20" style="font-weight: normal;"/>
        <input class="button" type="submit" value="Go!"/>
       </form>
      </div>
     </li>
     <li id="home">
      <a href="https://www.tilastopaja.info/">
       Home
      </a>
     </li>
     <li>
      <a href="#s1">
       Free
      </a>
      <span id="s1">
      </span>
      <ul class="subs">
       <li>
        <a href="https://www.tilastopaja.info/

In [375]:
# Traverse html with find_all and enumerate
# <TABLE> then <TR> then <TD> then <A> (name) or <B> (heats and stage)

lists=[]
counts=[]
values=[]
names=[]
wind=''
stage=''
event=''
gender=''
heat=''
dict_result2=[]  # length 2 dict
row=[]

temp_df = pd.DataFrame(columns=['RANK', 'NAME', 'NATIONALITY', 'RESULT', 'QUALIFICATION', 'COMPETITION', 'YEAR', 'DATE', 'EVENT', 'VENUE', 'GENDER', 'STAGE', 'HEAT', 'WIND', 'DOB', 'REMARKS', 'RX_TIME', 'DICT_RESULT'])

h1s = soup.find_all('h1')

for h1 in h1s:
    
    print(h1.text)


for i in soup.find_all('table'): # Look for the table first then find <tr> for each athlete
        
    for element in i.find_all('tr'): # <tr> separates each athlete/section
        
        print('NEW SECTION')
        
       # for td in element.find_all('td'):
    
        counts=[]
        values=[]
        
            
        for count, value in enumerate(element.find_all('td')):
            
            for n in value.find_all('a'):  # extract the name and put into a list
                
                if len(n.text)>5 and '(' not in n.text and '1y' not in n.text and '/' not in n.text:    # take out all the miscellaneous           
                    
                    names.append(n.text) # add 'clean' name to list
                              
            for m in value.find_all('b'):  # extract heats and stage
                                
                print('MMMMM', m.text)
                 
          #  for count_b, value_b in enumerate(value.find_all('b')):  # extract the name and put into a list
                                
          #      print('MMMMM', value_b.text)

                                       
            if len(lists)==2:  # captures gender
                
               # heat=''
                wind=''
                dict_result=''
                dict_result2=''
              #  stage=''
                
                print('RESET heat, stage HERE')
                
                if '4 x 100m' not in event and '4 x 400m' not in event:
                    
                    heat = ''
                    stage = ''
                
                if 'Men' in lists[1] and len(lists[1])==3: # exact match for 'Men'
                    gender = 'Male'
                    
                if 'Women' in lists[1]:
                    gender = 'Female'
                    
                     
            if 'Date' in value.text:
                date=value.text
                          
            if 'Wind' in value.text:
                wind=value.text
     #           print('WINDDDDD', value.text)
                
            print('TDTD', count, value.text) # extract Heat, Wind Rank, Nationality, DOB, Result, PB/SB
            
            counts.append(count)
            values.append(value.text)
            
                    
        lists=dict(zip(counts,values)) # convert every TD line under each TR section into a dictionary
        
        
        if len(lists)==3 and 'Wind' in lists[2]:  # if index 2 does not contain 'wind' then it stage
            
            heat = lists[1]
            wind = lists[2]  # wind can also appear in list of length 5
            
            if 'Wind' in lists[2]:
                
                wind = lists[2][5:]  # slice out the 'Wind:'
            
            
            dict_result=[]
            
        if len(lists)==3 and 'Wind' not in lists[2]:
            
            if 'Semifinals' in lists[1]:
                
                stage = 'Semifinals'
                
            if 'Heats' in lists[1]:
                
                stage = 'Heats'
                              
            if 'Heat 1' in lists[1]:
                
                heat = 'Heat 1'
                
            if 'Heat 2' in lists[1]:
                
                heat = 'Heat 2'
            
            if 'Heat 3' in lists[1]:
                
                heat = 'Heat 3'
            
            if 'Heat 4' in lists[1]:
                
                heat = 'Heat 4'
            
            
            if len(lists[2])>1:
                                
                date = lists[2]  # set date only if lists[2] is not null
            
            dict_result = []
                
        if len(lists)==5:  # dict of length=5 contains date, event info. Reset Heat, Stage and Wind when new event ecountered
            
            date = lists[2]
            event = lists[1]
            wind = ' '
            stage = ' '
            heat = ' '
            
            print('Reset STAGE, HEAT here')
            
            dict_result = []
            dict_result2 = []
            
            if 'Wind' in lists[4]:
                
                wind = lists[4][5:]  # slice out the 'Wind:'
            
                        
        if len(lists)==11: # dict length 10 contains athlete info
                        
            dict_result=lists
                        
            
                        
        #    dict_result2 = [] # reset attempts
            
                        
        if len(lists)==2 and 'Men' not in lists[1]: # dict length 2 contains event gender info or attempts info
            dict_result2 = lists
            
                    
        if len(lists)==2 and 'Women' not in lists[1]: # dict length 2 contains event gender info or attempts info
            dict_result2 = lists
        
        
        
        print('EVENT', event, 'GENDER', gender, 'STAGE', stage, 'HEAT', heat, 'WIND', wind, 'DATE', date, 'DICT_RESULT', dict_result, 'ATTEMPTS', dict_result2)
        
        try:
            
            if 'High Jump' in event or 'Pole Vault' in event or 'Long Jump' in event or 'Triple Jump' in event or 'Shot Put' in event or 'Discus Throw' in event or 'Hammer Throw' in event or 'Javelin Throw' in event or 'Decathlon' in event or '4 x 100m' in event or '4 x 400m' in event or 'Heptathlon' in event:
                print('dict_result2', dict_result2)
                
                if len(dict_result2)>0:
                          
                    
                    row.append(dict_result[0]) # rank
             #       for k in range(0, len(names)):  #search and replace name
                        
                    #    hh=''
                    
                   #     if names[k] in dict_result[1]:
                            
                   #         print('XXX', names[k], dict_result[1])
                            
                   #         hh=names[k]

                  #  row.append(hh) # name  (dict_result[1])
                    
                    if '4 x 100m' not in event and '4 x 400m' not in event and 'Mixed Relay' not in event:
                        
                        row.append(names[-1])
                    else:
                        row.append(' ')

                 #   row.append(dict_result[1]) # name  (dict_result[1])
                    row.append(dict_result[3]) # nationality
                    row.append(dict_result[5]) # result incl. DNS
                    row.append(dict_result[9]) # Q/q
                    row.append('18th Asian Games')
                    row.append('2018')
                    row.append(date)
                    row.append(event)
                    row.append('Gelora Bung Karno Stadium')
                    row.append(gender)
                    row.append(stage)
                    row.append(heat)
                    row.append(wind)
                    row.append(dict_result[4]) # DOB                    
                    row.append(dict_result[8]+' '+dict_result[2]) # Remarks. PB/SB
                    row.append(dict_result[10]) # Reaction time for 4 x 100m/400m 
                    row.append(dict_result2) # DICT_RESULT
                                        
                else:
                    
                    pass

                    
            
            elif (event and dict_result[3] and dict_result[5]) is not None:
                                    
               #

                row.append(dict_result[0]) # rank
                
                if '4 x 100m' not in event and '4 x 400m' not in event and 'Mixed Relay' not in event:
                        
                    row.append(names[-1])
                else:
                    row.append(' ')

                
             #   row.append(names[-1]) # name 
                
            #    for k in range(0, len(names)):  #search and replace name
                    
            #        hh=''
                    
            #        if names[k] in dict_result[1]:
                    
            #            hh=names[k]

             #   row.append(hh) # name  (dict_result[1])

               # row.append(dict_result[1]) # name  (dict_result[1])
                row.append(dict_result[3]) # nationality
                row.append(dict_result[5]) # result incl. DNS
                row.append(dict_result[9]) # Q/q
                row.append('18th Asian Games')
                row.append('2018')
                row.append(date)
                row.append(event)
                row.append('Gelora Bung Karno Stadium')
                row.append(gender)
                row.append(stage)
                row.append(heat)
                row.append(wind)
                row.append(dict_result[4]) # DOB                    
                row.append(dict_result[8]+' '+dict_result[2]) # Remarks. PB/SB
                row.append(dict_result[10]) # Reaction time for 4 x 100m/400m 
                row.append(dict_result2) # DICT_RESULT
                
               

            #print(row)
            
        except:
            
            pass
        
        if len(row)>5:
                    
            temp_df.loc[len(temp_df)] = row

        print('LENGTH', len(lists))

        
        dict_result2 = []
        row=[]
        
            
            
        
        
    wind=''
    heat=''
    date=''
    
    lists=[]
    dict_result = []
    row=[]
                        
            
                
            
            
            


        
      

TILASTOPAJA
RESULTS: 17th Asian Games, Incheon Asiad Main Stadium, Incheon, KOR  - 27 September - 3 October 2014 
LATEST MAJOR RESULTS
▼LATEST MAJOR RESULTS
NEW SECTION
TDTD 0 
 Men's events: 


Please choose men's event
Men  100m
Men  200m
Men  400m
Men  800m
Men  1500m
Men  5000m
Men  10,000m
Men  Marathon
Men  3000m Steeplechase
Men  110m Hurdles
Men  400m Hurdles
Men  High Jump
Men  Pole Vault
Men  Long Jump
Men  Triple Jump
Men  Shot Put
Men  Discus Throw
Men  Hammer Throw
Men  Javelin Throw
Men  Decathlon
Men  20km Race Walk
Men  50km Race Walk
Men  4 x 100m
Men  4 x 400m
Men Multievents 100m
Men Multievents 400m
Men Multievents 1500m
Men Multievents 110m Hurdles
Men Multievents High Jump
Men Multievents Pole Vault
Men Multievents Long Jump
Men Multievents Shot Put
Men Multievents Discus Throw
Men Multievents Javelin Throw



TDTD 1      
TDTD 2 
 Women's events: 


Please choose women's event
Women  100m
Women  200m
Women  400m
Women  800m
Women  1500m
Women  5000m
Women  10,000

LENGTH 11
NEW SECTION
TDTD 0 8
TDTD 1 Su Guoxiong Su Guoxiong  ('92)
TDTD 2 
TDTD 3 CHN
TDTD 4 26 May 92
TDTD 5 2:20:11 
TDTD 6 
TDTD 7  
TDTD 8 PB »
TDTD 9 
TDTD 10  
EVENT Marathon     GENDER Male STAGE   HEAT   WIND   DATE 3 October DICT_RESULT {0: '8', 1: "Su Guoxiong\xa0Su Guoxiong\xa0\xa0('92)", 2: '', 3: 'CHN', 4: '26 May 92', 5: '2:20:11 ', 6: '', 7: '\xa0', 8: 'PB »', 9: '', 10: ' '} ATTEMPTS []
LENGTH 11
NEW SECTION
TDTD 0 9
TDTD 1 Gantulga DambadarjaaGantulga Dambadarjaa ('89)
TDTD 2 
TDTD 3 MGL
TDTD 4 3 Feb 89
TDTD 5 2:20:54 
TDTD 6 
TDTD 7  
TDTD 8 PB »
TDTD 9 
TDTD 10  
EVENT Marathon     GENDER Male STAGE   HEAT   WIND   DATE 3 October DICT_RESULT {0: '9', 1: "Gantulga\xa0DambadarjaaGantulga\xa0Dambadarjaa\xa0('89)", 2: '', 3: 'MGL', 4: '3 Feb 89', 5: '2:20:54 ', 6: '', 7: '\xa0', 8: 'PB »', 9: '', 10: ' '} ATTEMPTS []
LENGTH 11
NEW SECTION
TDTD 0 10
TDTD 1 Shim Jong-Sub Shim Jong-Sub  ('91)
TDTD 2 
TDTD 3 KOR
TDTD 4 21 Feb 91
TDTD 5 2:23:11 
TDTD 6 
TDTD 7  
TDTD 8 « »


LENGTH 11
NEW SECTION
TDTD 0 6
TDTD 1 Cai Minjia Cai Minjia  ('93)
TDTD 2 
TDTD 3 CHN
TDTD 4 2 Apr 93
TDTD 5 59.78 
TDTD 6 
TDTD 7  
TDTD 8 « »
TDTD 9 
TDTD 10 0.213     
EVENT 400m Hurdles     GENDER Female STAGE   HEAT   WIND   DATE 1 October DICT_RESULT {0: '6', 1: "Cai Minjia\xa0Cai Minjia\xa0\xa0('93)", 2: '', 3: 'CHN', 4: '2 Apr 93', 5: '59.78 ', 6: '', 7: '\xa0', 8: '« »', 9: '', 10: '0.213     '} ATTEMPTS []
LENGTH 11
NEW SECTION
TDTD 0 7
TDTD 1 Jo Eun-Ju Jo Eun-Ju  ('89)
TDTD 2 
TDTD 3 KOR
TDTD 4 12 Nov 89
TDTD 5 59.89 
TDTD 6 
TDTD 7  
TDTD 8 « »
TDTD 9 
TDTD 10 0.207     
EVENT 400m Hurdles     GENDER Female STAGE   HEAT   WIND   DATE 1 October DICT_RESULT {0: '7', 1: "Jo Eun-Ju\xa0Jo Eun-Ju\xa0\xa0('89)", 2: '', 3: 'KOR', 4: '12 Nov 89', 5: '59.89 ', 6: '', 7: '\xa0', 8: '« »', 9: '', 10: '0.207     '} ATTEMPTS []
LENGTH 11
NEW SECTION
TDTD 0 8
TDTD 1 Natalya AsanovaNatalya Asanova ('89)
TDTD 2 
TDTD 3 UZB
TDTD 4 29 Nov 89
TDTD 5 1:00.42 
TDTD 6 
TDTD 7  
TDTD 8 « »
TDTD 9 

In [376]:
len(names)

982

In [377]:
names

['Femi\xa0Ogunode',
 'Su Bingtian\xa0',
 'Kei\xa0Takase',
 'Zhang Peimeng\xa0',
 'Reza\xa0Ghasemi',
 'Ryota\xa0Yamagata',
 'Barakat\xa0Al-Harthi',
 'Samuel\xa0Francis',
 'Ryota\xa0Yamagata',
 'Su Bingtian\xa0',
 'Samuel\xa0Francis',
 'Hassan\xa0Taftian',
 'Kim Kuk-Young\xa0',
 'Tsui Chi Ho\xa0',
 'Jirapong\xa0Meenapra',
 'Hassan\xa0Saaid',
 'Femi\xa0Ogunode',
 'Kei\xa0Takase',
 'Zhang Peimeng\xa0',
 'Reza\xa0Ghasemi',
 'Barakat\xa0Al-Harthi',
 'Oh Kyung-Su\xa0',
 'Ruttanapon\xa0Sowan',
 'Mohd Amiruddin\xa0Jamal',
 'Ryota\xa0Yamagata',
 'Zhang Peimeng\xa0',
 'Jirapong\xa0Meenapra',
 'Tsui Chi Ho\xa0',
 'Meshal\xa0Al-Mutairi',
 'Pao Hin Fong\xa0',
 'Samuel\xa0Francis',
 'Su Bingtian\xa0',
 'Ruttanapon\xa0Sowan',
 'Mohd Amiruddin\xa0Jamal',
 'Abbos\xa0Toshmatov',
 'So Chun Hong\xa0',
 'Kei\xa0Takase',
 'Kim Kuk-Young\xa0',
 'Reza\xa0Ghasemi',
 'Hassan\xa0Saaid',
 'Achitbileg\xa0Battulga',
 'Mohammed\xa0Abukhousa',
 'Leong Wang Kuong\xa0',
 'Femi\xa0Ogunode',
 'Barakat\xa0Al-Harthi',
 'Has

In [378]:
namess=pd.DataFrame(names)

In [379]:
namess.to_csv('names.csv', encoding='utf-8')

In [380]:
temp_df

,RANK,NAME,NATIONALITY,RESULT,QUALIFICATION,COMPETITION,YEAR,DATE,EVENT,VENUE,GENDER,STAGE,HEAT,WIND,DOB,REMARKS,RX_TIME,DICT_RESULT
0,1,Femi Ogunode,QAT,9.93,,18th Asian Games,2018,28 September,100m,Gelora Bung Karno Stadium,Male,,,0.4,15 May 91,PB »,0.184,[]
1,2,Su Bingtian,CHN,10.10,,18th Asian Games,2018,28 September,100m,Gelora Bung Karno Stadium,Male,,,0.4,29 Aug 89,SB «,0.145,[]
2,3,Kei Takase,JPN,10.15,,18th Asian Games,2018,28 September,100m,Gelora Bung Karno Stadium,Male,,,0.4,25 Nov 88,« »,0.152,[]
3,4,Zhang Peimeng,CHN,10.18,,18th Asian Games,2018,28 September,100m,Gelora Bung Karno Stadium,Male,,,0.4,13 Mar 87,« »,0.128,[]
4,5,Reza Ghasemi,IRI,10.25,,18th Asian Games,2018,28 September,100m,Gelora Bung Karno Stadium,Male,,,0.4,24 Jul 87,SB «,0.161,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,2,Huang Yu-Ting,TPE,14.36,,18th Asian Games,2018,28 September,100m Hurdles,Gelora Bung Karno Stadium,Female,Heats,Heat 2,-0.5,17 Nov 88,« »,0.239,[]
931,3,Kotchakorn Khamrueangsri,THA,14.52,,18th Asian Games,2018,28 September,100m Hurdles,Gelora Bung Karno Stadium,Female,Heats,Heat 2,-0.5,13 Mar 95,« »,0.222,[]
932,4,Sushmita Singha Roy,IND,14.67,,18th Asian Games,2018,28 September,100m Hurdles,Gelora Bung Karno Stadium,Female,Heats,Heat 2,-0.5,26 Mar 84,« »,0.229,[]
933,5,Yekaterina Voronina,UZB,15.19,,18th Asian Games,2018,28 September,100m Hurdles,Gelora Bung Karno Stadium,Female,Heats,Heat 2,-0.5,16 Feb 92,PB »,0.192,[]


In [381]:
columns =  list(temp_df.columns) 
    
for col in columns:
    temp_df[col] = temp_df[col].astype(str)
    temp_df[col] = temp_df[col].str.replace('\xa0', ' ', regex=True)
    temp_df[col] = temp_df[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    
    

In [382]:
temp_df.to_csv("2017_AG.csv", encoding='utf-8')